In [ ]:
pip install split-folders

In [15]:
import matplotlib.pyplot as plt
import splitfolders
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Splits the data file into train, validate and test 

In [11]:
# this splits the data folder into a train, test and validate 
splitfolders.ratio('Data', output="output", seed=1337, ratio=(0.7, 0.2,0.1)) 


Copying files: 12437 files [00:28, 441.84 files/s]


In [28]:
train_image_folder = 'data/train/'
val_image_folder = 'data/val/'
test_image_folder = 'data/test/'

In [38]:
# Instantiating our network
model_1 = Sequential()

# Adding layers
model_1.add(Dense(64, activation = 'relu', input_shape=(8705,)))
model_1.add(Dense(64, activation = 'relu'))
model_1.add(Dense(2, activation='softmax'))

# Compiling 
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# Displaying the network structure
model_1.summary()

# Fitting to the train data
model_1_results = model_1.fit(train_image_folder, 
                                epochs = 50, 
                                batch_size = 100,
                                validation_data = val_image_folder,
                                verbose = 1)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 64)                557184    
_________________________________________________________________
dense_12 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 130       
Total params: 561,474
Trainable params: 561,474
Non-trainable params: 0
_________________________________________________________________


IndexError: list index out of range

sample of augumented image

In [26]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/sold_nft_images/nft-image-546.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x=x, batch_size=1, save_prefix='nft', save_format='jpg'):
    i += 1
    if i > 10:
        break 

In [25]:
# the output of the image is in the folder preview

### Network

In [29]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [33]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [30]:
image_generator = ImageDataGenerator(rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

In [31]:
# frm: https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/

# load and iterate training dataset
train_it = image_generator.flow_from_directory(train_image_folder, class_mode='binary',
                                       batch_size=64)
# load and iterate validation dataset
val_it = image_generator.flow_from_directory(val_image_folder, class_mode='binary',
                                     batch_size=64)
# load and iterate test dataset
test_it = test_datagen.flow_from_directory(test_image_folder, class_mode='binary',
                                      batch_size=64)

Found 8705 images belonging to 2 classes.
Found 2487 images belonging to 2 classes.
Found 1245 images belonging to 2 classes.


In [34]:
model.fit_generator(train_it, steps_per_epoch=16,
                    validation_data=val_it, validation_steps=8)

DecompressionBombError: Image size (245840445 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.

In [ ]:
model.save_weights('first_try.h5')

In [ ]:
# first model